Solver

In [2]:
using JuMP
using Gurobi
using CSV
using DataFrames

Data Importation

In [34]:
budgets_demand = CSV.File("./data/budgets_demand.csv",header=0) |> Tables.matrix;
impression_data = CSV.File("./data/impression_per_age.csv",header=0) |> Tables.matrix; # plaform, age group, impressions rate
company_rcs_data = CSV.File("./data/company_rcs.csv",header=0) |> Tables.matrix; # company, rps
platform_variables_data = CSV.File("./data/platform_variables.csv",header=0) |> Tables.matrix; # platform, variable, value


Clean Data

In [50]:
# define dictionaries to use

# B: budget
budget_dict = Dict() # dict of (company) -> budget; 50 companies
for i in 2:size(budgets_demand,1)
    budget_dict[budgets_demand[i, 1]] = parse(Int, budgets_demand[i, 2])
end

# R_a: total reach demanded for each age group
demand_dict = Dict() # dict of (company, age group) -> demand; 50 companies, 4 age groups
age_groups = ["18-24", "25-34", "35-44", "45-60"]
for i in 2:size(budgets_demand,1)
    for j in 1:4
        demand_dict[(budgets_demand[i, 1], demands[1,j])] = parse(Int, demands[i,j])
    end
end

# impression rate per age group (a_ap*i_p)
impressions_per_age = Dict() # dict of (platform, age group) -> impression rate
for i in 2:size(impression_data,1)
    str_value = string(impression_data[i, 3])  # Convert to regular String
    impressions_per_age[(impression_data[i, 1], impression_data[i, 2])] = parse(Float64, str_value)  # Parse to Int
end


# company revenue per conversion
company_rcs = Dict() # dict of company -> r_c
for i in 1:size(company_rcs_data,2)
    # row 1 is company name, row 2 is r_c
    company_rcs[company_rcs_data[1, i]] = parse(Float64, company_rcs_data[2, i])
end

# platform variables
# constant for all companies need for each platform, 
    # w_p: click_rate
    # i_p: impression_rate
    # f_p: average_impression_rate
    # c_p: conversion rate
    # s_p: cost per click 

platform_variables = Dict() # dict of (platform, variable) -> value
for i in 2:size(platform_variables_data, 2)
    for j in 2:size(platform_variables_data, 1)
        platform_variables[platform_variables_data[1, i], platform_variables_data[j, 1]] = parse(Float64, platform_variables_data[j, i])
    end
end

